In [ ]:
import pandas as pd
import numpy as np
from scipy.stats.mstats import winsorize

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.inspection import permutation_importance
from imblearn.over_sampling import SMOTE

# ---------------------------
# CONFIG
# ---------------------------
TRAIN_PATH = "train.csv"
TEST_PATH = "test.csv"
TARGET = "has_copd_risk"
RANDOM_STATE = 42

# ---------------------------
# LOAD
# ---------------------------
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

# ---------------------------
# OUTLIER HANDLING (winsorize)
# ---------------------------
df_clean = train.copy()

numeric_cols_raw = df_clean.select_dtypes(include=["float64", "int64"]).columns.tolist()
numeric_cols_raw = [c for c in numeric_cols_raw if c not in ["patient_id"]]

for col in numeric_cols_raw:
    # small limits to clip extreme outliers
    df_clean[col] = winsorize(df_clean[col], limits=[0.01, 0.01])

# ---------------------------
# PREPROCESS: one-hot encode and target mapping
# ---------------------------
df = df_clean.copy()

# identify categorical columns (object dtype)
cat_cols = df.select_dtypes(include=["object"]).columns.tolist()
cat_cols = [c for c in cat_cols if c != TARGET]

df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

# map target Y/N to 1/0 if necessary
if df[TARGET].dtype == "object":
    df[TARGET] = df[TARGET].map({"Y": 1, "N": 0})

# prepare X,y (we will later expand X with engineered features)
y = df[TARGET].astype(int)
X = df.drop(columns=[TARGET])

# fill missing numeric values (column means)
X = X.fillna(X.mean())

# ---------------------------
# BASELINE IMPORTANCE COMPUTATION (no PCA)
# ---------------------------
# scale baseline X for model training
scaler_for_importance = StandardScaler()
X_scaled_for_imp = scaler_for_importance.fit_transform(X)

# split for validation to compute permutation importance
X_train0, X_val0, y_train0, y_val0 = train_test_split(
    X_scaled_for_imp, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

print("Training quick baseline models for feature importance...")

# quick baseline models (smaller training iterations to speed up)
logreg0 = LogisticRegression(max_iter=500, class_weight="balanced", random_state=RANDOM_STATE)
logreg0.fit(X_train0, y_train0)

bayes0 = GaussianNB()
bayes0.fit(X_train0, y_train0)

svm0 = SVC(kernel="rbf", class_weight="balanced", probability=False, random_state=RANDOM_STATE)
svm0.fit(X_train0, y_train0)

mlp0 = MLPClassifier(hidden_layer_sizes=(32, 16), max_iter=150, random_state=RANDOM_STATE)
mlp0.fit(X_train0, y_train0)

feature_names = X.columns.tolist()

# Helper importance functions
def logreg_importance(model, feature_names):
    coef = np.abs(model.coef_[0])
    dfi = pd.DataFrame({"feature": feature_names, "importance": coef})
    return dfi.sort_values("importance", ascending=False).set_index("feature")["importance"]

def bayes_importance(model, feature_names):
    # GaussianNB sigma_ shape: (n_classes, n_features) -> use class 1 if binary
    # Use inverse variance (1 / sigma) as simple importance proxy
    # If sigma_ is zero (rare), add small eps
    sigma = model.var_
    if sigma.ndim == 2:
        sigma_class = sigma[1] if sigma.shape[0] > 1 else sigma[0]
    else:
        sigma_class = sigma
    importance = 1.0 / (sigma_class + 1e-8)
    dfi = pd.DataFrame({"feature": feature_names, "importance": importance})
    return dfi.sort_values("importance", ascending=False).set_index("feature")["importance"]

def perm_importance_wrapper(model, X_val, y_val, feature_names, n_repeats=5):
    # returns series indexed by feature name
    res = permutation_importance(model, X_val, y_val, scoring="accuracy", n_repeats=n_repeats, random_state=RANDOM_STATE, n_jobs=-1)
    dfi = pd.DataFrame({"feature": feature_names, "importance": res.importances_mean})
    return dfi.sort_values("importance", ascending=False).set_index("feature")["importance"]

# compute importances
logreg_imp = logreg_importance(logreg0, feature_names)
bayes_imp = bayes_importance(bayes0, feature_names)
svm_imp = perm_importance_wrapper(svm0, X_val0, y_val0, feature_names, n_repeats=5)
mlp_imp = perm_importance_wrapper(mlp0, X_val0, y_val0, feature_names, n_repeats=5)

# align and aggregate (mean importance across models)
imps_df = pd.concat([
    logreg_imp.rename("logreg"),
    bayes_imp.rename("bayes"),
    svm_imp.rename("svm"),
    mlp_imp.rename("mlp")
], axis=1).fillna(0)

imps_df["agg_mean"] = imps_df.mean(axis=1)
imps_df = imps_df.sort_values("agg_mean", ascending=False)

print("\nTop 20 features by aggregated importance:")
print(imps_df["agg_mean"].head(20))

# choose top features for engineering
TOP_K = 10
top_features = imps_df.index[:TOP_K].tolist()
print(f"\nSelected top {TOP_K} features for feature engineering: {top_features}")

# ---------------------------
# FEATURE ENGINEERING FUNCTIONS
# ---------------------------
def add_interactions_df(df_in, features, max_pairs=28):
    """Add pairwise multiplication interactions among provided features.
       Limit to first max_pairs combinations to avoid explosion."""
    df = df_in.copy()
    combos = []
    n = len(features)
    for i in range(n):
        for j in range(i+1, n):
            combos.append((features[i], features[j]))
    # limit combos deterministically
    combos = combos[:max_pairs]
    for f1, f2 in combos:
        new_name = f"{f1}_x_{f2}"
        df[new_name] = df[f1] * df[f2]
    return df

def add_polynomials_df(df_in, features, degree=2):
    df = df_in.copy()
    for f in features:
        if degree >= 2:
            df[f"{f}_sq"] = df[f] ** 2
    return df

def add_ratios_df(df_in, features, denom_count=4):
    df = df_in.copy()
    base = features[0]
    for f in features[1:1+denom_count]:
        new_name = f"{base}_ratio_{f}"
        df[new_name] = df[base] / (df[f] + 1e-8)
    return df

# ---------------------------
# APPLY ENGINEERING TO TRAIN DF
# ---------------------------
df_enh = df.copy()  # df currently includes X + target handled earlier (we will drop target when needed)

# add interactions (limit pairs to avoid too many features)
df_enh = add_interactions_df(df_enh, top_features, max_pairs=28)
df_enh = add_polynomials_df(df_enh, top_features, degree=2)
df_enh = add_ratios_df(df_enh, top_features, denom_count=4)

# ensure no infinite / NaN values
df_enh = df_enh.replace([np.inf, -np.inf], np.nan).fillna(0)

# ---------------------------
# RECOMPUTE IMPORTANCE ON ENHANCED SET (OPTIONAL QUICK CHECK)
# ---------------------------
# Re-derive X,y from enhanced df
y_enh = df_enh[TARGET].astype(int)
X_enh = df_enh.drop(columns=[TARGET])

# Fill missing and scale
X_enh = X_enh.fillna(X_enh.mean())
scaler_enh = StandardScaler()
X_enh_scaled = scaler_enh.fit_transform(X_enh)

# Quick logistic to get coefficients for pruning guidance
logreg_check = LogisticRegression(max_iter=500, class_weight="balanced", random_state=RANDOM_STATE)
logreg_check.fit(X_enh_scaled, y_enh)

coef_abs = np.abs(logreg_check.coef_[0])
coef_series = pd.Series(coef_abs, index=X_enh.columns).sort_values(ascending=False)

# ---------------------------
# PRUNE LOW-IMPORTANCE FEATURES
# ---------------------------
# Use aggregated importance from earlier as baseline; for any engineered features not present,
# compute use coefficient magnitude as proxy. Build a combined importance Series.
agg_importance = imps_df["agg_mean"].copy()

# engineered features may not be in agg_importance; map them via coefficient magnitude (normalized)
coef_norm = coef_series / (coef_series.max() + 1e-12)

combined_imp = agg_importance.reindex(X_enh.columns).fillna(0).copy()
# where combined_imp is zero (new features), fill with coef_norm scaled to mean of agg_importance
mean_agg = agg_importance.mean() if agg_importance.shape[0] > 0 else 0.0
combined_imp = combined_imp + coef_norm * (mean_agg if mean_agg > 0 else 0.01)

# normalize to sum to 1 for interpretable thresholding
combined_imp_norm = combined_imp / (combined_imp.sum() + 1e-12)

# drop features with < threshold importance (relative)
PRUNE_THRESHOLD = 0.001  # drop features contributing less than 0.1% of total importance
low_features = combined_imp_norm[combined_imp_norm < PRUNE_THRESHOLD].index.tolist()
print(f"\nPruning {len(low_features)} features with combined importance < {PRUNE_THRESHOLD}")
X_pruned = X_enh.drop(columns=low_features, errors="ignore")

print(f"Features remaining after prune: {X_pruned.shape[1]}")

# ---------------------------
# FINAL X,y, SCALE and PIPELINE
# ---------------------------
X_final = X_pruned.copy()
y_final = y_enh.copy()

# scale for downstream PCA and models
scaler = StandardScaler()
X_final_scaled = scaler.fit_transform(X_final)

# ---------------------------
# TRAINING FUNCTION (uses PCA + SMOTE for MLP)
# ---------------------------
def train_models(X_scaled_all, y_all, test_size, n_components=None):
    X_train, X_val, y_train, y_val = train_test_split(
        X_scaled_all, y_all, test_size=test_size, random_state=RANDOM_STATE, stratify=y_all
    )

    print(f"\n======== Test Size = {test_size} ========")

    # PCA
    pca = PCA(n_components=n_components or 0.95, random_state=RANDOM_STATE)
    X_train_pca = pca.fit_transform(X_train)
    X_val_pca = pca.transform(X_val)

    print(f"PCA reduced {X_train.shape[1]} → {X_train_pca.shape[1]} features")

    # SMOTE (MLP only)
    sm = SMOTE(random_state=RANDOM_STATE)
    X_train_bal, y_train_bal = sm.fit_resample(X_train_pca, y_train)

    # ---------------- MLP ----------------
    mlp = MLPClassifier(
        hidden_layer_sizes=(32, 16),
        activation="relu",
        solver="adam",
        learning_rate_init=0.001,
        alpha=0.0005,
        batch_size=64,
        early_stopping=True,
        max_iter=400,
        n_iter_no_change=15,
        random_state=RANDOM_STATE
    )
    mlp.fit(X_train_bal, y_train_bal)
    mlp_acc = accuracy_score(y_val, mlp.predict(X_val_pca))

    # ---------------- LogReg ----------------
    logreg = LogisticRegression(max_iter=500, class_weight="balanced", random_state=RANDOM_STATE)
    logreg.fit(X_train_pca, y_train)
    logreg_acc = accuracy_score(y_val, logreg.predict(X_val_pca))

    # ---------------- Bayes ----------------
    bayes = GaussianNB()
    bayes.fit(X_train_pca, y_train)
    bayes_acc = accuracy_score(y_val, bayes.predict(X_val_pca))

    # ---------------- SVM GridSearch ----------------
    svm_param_grid = {
        "C": [0.3, 1, 3, 10],
        "gamma": ["scale", "auto", 1e-4, 3e-4, 1e-3],
        "kernel": ["rbf"]
    }

    svm = SVC(class_weight="balanced", random_state=RANDOM_STATE)

    svm_cv = GridSearchCV(
        svm,
        svm_param_grid,
        scoring="accuracy",
        cv=3,
        n_jobs=-1,
        verbose=0
    )
    svm_cv.fit(X_train_pca, y_train)
    svm_best = svm_cv.best_estimator_

    svm_acc = accuracy_score(y_val, svm_best.predict(X_val_pca))

    print(f"MLP Acc     = {mlp_acc:.4f}")
    print(f"LogReg Acc  = {logreg_acc:.4f}")
    print(f"Bayes Acc   = {bayes_acc:.4f}")
    print(f"SVM Acc     = {svm_acc:.4f}")
    print(f"Best SVM Params: {svm_cv.best_params_}")

    return mlp, logreg, bayes, svm_best, pca

# ---------------------------
# TRAIN for two test sizes (like your original)
# ---------------------------
mlp_02, logreg_02, bayes_02, svm_02, pca_02 = train_models(X_final_scaled, y_final, test_size=0.2)
mlp_08, logreg_08, bayes_08, svm_08, pca_08 = train_models(X_final_scaled, y_final, test_size=0.8)

# ---------------------------
# PREPROCESS TEST SET: apply same feature engineering and pruning
# ---------------------------
test_df = test.copy()

# Apply same get_dummies for categorical columns we used originally.
# Note: original cat_cols were derived from train before get_dummies; ensure same columns here.
test_df = pd.get_dummies(test_df, columns=cat_cols, drop_first=True)

# Reindex to include any missing columns from training (X_final columns + the pruned ones we dropped)
# First, create engineered features on test_df using the same functions and top_features
# BUT because test_df may be missing some original dummy columns, reindex below to full set with zeros.

# Add engineered features to test_df (they will reference columns from test that may not exist yet;
# we will create missing columns with zeros after reindexing)
test_df = add_interactions_df(test_df, top_features, max_pairs=28)
test_df = add_polynomials_df(test_df, top_features, degree=2)
test_df = add_ratios_df(test_df, top_features, denom_count=4)

# Replace infinities and NaNs
test_df = test_df.replace([np.inf, -np.inf], np.nan).fillna(0)

# Reindex test_df to have exactly the columns of X_enh (before pruning) but missing columns will be filled with 0
test_df = test_df.reindex(columns=X_enh.columns, fill_value=0)

# Now apply pruning: remove the features we dropped (low_features)
test_df_pruned = test_df.drop(columns=low_features, errors="ignore")

# Final feature order should match X_final.columns
test_df_pruned = test_df_pruned.reindex(columns=X_final.columns, fill_value=0)

# scale using the same scaler used for training (scaler variable)
test_scaled = scaler.transform(test_df_pruned)

# transform via PCA (two pca models)
test_pca_02 = pca_02.transform(test_scaled)
test_pca_08 = pca_08.transform(test_scaled)

# ---------------------------
# SAVE RESULTS (same save function)
# ---------------------------
def save_output(filename, labels):
    pd.DataFrame({
        "patient_id": test["patient_id"],
        "has_copd_risk": labels
    }).to_csv(filename, index=False)

save_output("copd_predictions_mlp_02.csv", mlp_02.predict(test_pca_02))
save_output("copd_predictions_logreg_02.csv", logreg_02.predict(test_pca_02))
save_output("copd_predictions_bayes_02.csv", bayes_02.predict(test_pca_02))
save_output("copd_predictions_svm_02.csv", svm_02.predict(test_pca_02))

save_output("copd_predictions_mlp_08.csv", mlp_08.predict(test_pca_08))
save_output("copd_predictions_logreg_08.csv", logreg_08.predict(test_pca_08))
save_output("copd_predictions_bayes_08.csv", bayes_08.predict(test_pca_08))
save_output("copd_predictions_svm_08.csv", svm_08.predict(test_pca_08))

print("\nAll predictions saved.")

Training quick baseline models for feature importance...

Top 20 features by aggregated importance:
feature
sex_M                     1.922985
hemoglobin_level          0.509274
height_cm                 0.502163
serum_creatinine          0.387072
bp_systolic               0.330507
weight_kg                 0.318126
hearing_left              0.309947
tartar_presence_Y         0.306049
waist_circumference_cm    0.295470
triglycerides             0.295104
hearing_right             0.293218
ggt_enzyme_level          0.291284
hdl_cholesterol           0.287323
age_group                 0.285846
bp_diastolic              0.285536
vision_right              0.267097
vision_left               0.265859
total_cholesterol         0.263803
ldl_cholesterol           0.252161
patient_id                0.251736
Name: agg_mean, dtype: float64

Selected top 10 features for feature engineering: ['sex_M', 'hemoglobin_level', 'height_cm', 'serum_creatinine', 'bp_systolic', 'weight_kg', 'hearing_left', 'ta